In [1]:
import codecs
import os
from collections import deque
import re
import random
from itertools import zip_longest

In [2]:
data_dir = '/home/disooqi/RBGParser/data'

train_path = os.path.join(data_dir, 'spmrl.train.conll')
test_path = os.path.join(data_dir, 'spmrl.test.conll')

tokfile = os.path.join(data_dir, 'spmrl_train_conll.tok')

farasa_output = os.path.join(data_dir, 'for_aligning')
RBGParser_train = os.path.join(data_dir, 'arabic.train.farasa')

In [3]:
ar = "\u0627\u0625\u0622\u0623\u0621\u0628\u062a\u062b\u062c\u062d\u062e\u062f\u0630\u0631\u0632\u0633\u0634\u0635\u0636\u0637\u0638\u0639\u063a\u0641\u0642\u0643\u0644\u0645\u0646\u0647\u0648\u064a\u0649\u0629\u0624\u0626\u064e\u064b\u064f\u064c\u0650\u064d\u0652\u0651"
buck = "A<|>'btvjHxd*rzs$SDTZEgfqklmnhwyYp&}aFuNiKo~"
b2a_translation_table = str.maketrans(buck, ar)
a2b_translation_table = str.maketrans(ar, buck)

def utf82buck(kdinput):
    return kdinput.translate(a2b_translation_table)

In [4]:
p = re.compile(r'\.\.+')
p2 = re.compile(r';')
p3 = re.compile(r'/')
p4 = re.compile(r'_')
fp= re.compile(r'\d+\.\d+')
fp2= re.compile(r'\d+\.\d+\.\d+')
fp3= re.compile(r'\.\d+')
ap= re.compile(r'\d+,\d+')
with codecs.open(train_path, encoding='utf-8') as tf:
    with codecs.open(tokfile, encoding='utf-8', mode='w') as tt:
        for line in tf:
            fields = line.strip().split()
            
            if len(fields) == 10:
                sentence = line.strip().split()[1]
                sentence = p.sub('.',sentence)
                sentence = p2.sub(',',sentence)
                sentence = p3.sub(',',sentence)
                sentence = p4.sub(',',sentence)
                sentence = ap.sub(str(random.randrange(1,1000)), sentence)
                sentence = fp2.sub(str(random.randrange(1,1000)), sentence)
                sentence = fp.sub(str(random.randrange(1,1000)), sentence)
                sentence = fp3.sub(str(random.randrange(1,1000)), sentence)
                
                tt.write(sentence)
            else:
                tt.write(line.strip())
            tt.write('\n')

In [5]:
col2 = list()
col7 = list()
col8 = list()
col9 = list()
col10 = list()

with codecs.open(train_path, encoding='utf-8') as tf:
    for line in tf:
            fields = line.strip().split()
            if len(fields) == 10:
                col2.append(fields[1])
                col7.append(fields[6])
                col8.append(fields[7])
                col9.append(fields[8])
                col10.append(fields[9])
            else:
                pass

In [11]:
from collections import Counter

print(list(Counter(col8)))
len(col2), len(col7),len(col8),len(col9),len(col10)

['OBJ', 'PRD', 'TMZ', 'IDF', '---', 'SBJ', 'MOD', 'TPC']


(451280, 451280, 451280, 451280, 451280)

In [ ]:
col2_q = deque(col2[:])
col7_q = deque(col7[:])
col8_q = deque(col8[:])
col9_q = deque(col9[:])
col10_q = deque(col10[:])

with codecs.open(farasa_output, encoding='utf-8') as tf:
    with codecs.open(RBGParser_train, encoding='utf-8', mode='w') as tt:
        count = 0 
        for org_line in tf:
            org_tokens = org_line.strip().split()
            
            farasa_line = tf.readline()
            farasa_tokens = farasa_line.strip().split()[1:-1]
            #print(len(org_tokens), len(farasa_tokens), farasa_tokens[0], farasa_tokens[-1])
            #farasa_q = deque(farasa_tokens)
            # sentence = list()
            if not len(org_tokens) == len(farasa_tokens):
                count += 1
                print ("the 2 sentences are not equal", count)
                print (org_line)
                print(len(org_tokens), len(farasa_tokens), farasa_tokens[0], farasa_tokens[-1])
                #print(org_line)
                #print(farasa_line)
                print([print(tup) for tup in list(zip_longest(org_tokens, farasa_tokens))])
                print()
                
            for ID, token in enumerate(zip(org_tokens, farasa_tokens)):
                #print('\n')
                #print(token[0], token[1])
                #print('='*40)
                
                fields = list()
                feats = {'det':'n', 'gen':'na', 'num':'na'}
                fields.append(str(ID+1))
                fields.append(col2_q.popleft())
                # fields.append(token[0])
                # first = farasa_q.popleft().split('/')
                # print(token[0], token[1])
                
                tmp = token[1].split('|')
                if len(tmp) > 1:
                    pos = ''
                    for t in tmp:
                        tok_pos = t.split('/')
                        if '+' not in tok_pos[0]:
                            fields.append(utf82buck(tok_pos[0]))
                            pos_gen_num = tok_pos[1].split('-')
                            if len(pos_gen_num) == 2:
                                fields.append(pos_gen_num[0])
                                fields.append(pos_gen_num[0])
                                
                                feats['gen'] = pos_gen_num[1][:1]
                                feats['num'] = pos_gen_num[1][1:]
                            else:
                                fields.append(pos_gen_num[0])
                                fields.append(pos_gen_num[0])

                        elif tok_pos[0] == 'ال+':
                            feats['det'] = 'y'
                        else:
                            pos += '+'+tok_pos[1]
                    else:
                        if len(fields) != 5: # at this moment
                            fields.append(fields[1])
                            fields.append(pos[1:])
                            fields.append(pos[1:])
                            #print(len(fields), fields)
                            
                else:                    
                    feats['det'] = 'n'
                    tok_pos = tmp[0].split('/')
                    if '+' in tok_pos[0]:
                        tok_pos[0] = tok_pos[0].replace('+', '')
                        
                    fields.append(utf82buck(tok_pos[0]))
                    pos_gen_num = tok_pos[1].split('-')
                    if len(pos_gen_num) == 2:
                        fields.append(pos_gen_num[0])
                        fields.append(pos_gen_num[0])
                        feats['gen'] = pos_gen_num[1][:1]
                        feats['num'] = pos_gen_num[1][1:]
                    else:
                        fields.append(pos_gen_num[0])
                        fields.append(pos_gen_num[0])

                        
                fields.append('det='+feats['det']+'|gen='+feats['gen']+'|num='+feats['num'])
                fields.append(col7_q.popleft())
                fields.append(col8_q.popleft())
                fields.append(col9_q.popleft())
                fields.append(col10_q.popleft())
                
                fis = '\t'.join(fields)
                offf = len(fis.strip().split())
           
                if offf != 10:
                    print(fis)
                
                tt.write('\t'.join(fields))
                tt.write('\n')
            tt.write('\n')
            
                # print('\t'.join(fields))
            #break
            tf.readline()
            tf.readline() # just consuming empty line
 

In [ ]:
d = 'ال+/DETمتحد/ADJ-MP+ة/NSUFF'
d.split('+')